In [ ]:
#Dependencies
import pandas as pd
import datetime 
from pprint import pprint
import json
from sqlalchemy import create_engine

# Extract CSVs into DataFrames

We got our data from https://www.kaggle.com/datasnaek/youtube-new. We chose to work with the CSV and JSON files for Canada. CSV was easy to extract, the JSON file required a few additional steps.

In [ ]:
# Importing csv file

csv_file = "Resources/CA_youtube_trending_data.csv"
canada = pd.read_csv(csv_file)
canada

In [ ]:
# importing json file

json_file = "Resources/CA_category_id.json"

In [ ]:
# load json file as 'data'

with open(json_file) as file:
    data = json.load(file)
pprint(data)

In [ ]:
# put 'data' into lists

category_id = []
channel_id = []
title = []

for item in data["items"]:
    category_id.append(item['id'])
    channel_id.append(item['snippet']['channelId'])
    title.append(item['snippet']['title'])
    
title


In [ ]:
#  create dictionary from the lists then to dataframe

category_dict = { "category_id": category_id,
                 "category_title":title
}

category_df = pd.DataFrame(category_dict)
category_df

# Transform Dataframes

In the transform section we will clean, amend, and select specific columns and rows that we want to work with. We will also be merging, aggregating, and filtering the dataframes. At the end, we will create a few new tables out of the original two based on the end objective of our project. 

### Canada DataFrame

In [ ]:
# select the columns we want to work with from CSV

canada_one = canada[["video_id", "title", "publishedAt", "channelTitle", "categoryId",
                    "view_count", "likes", "dislikes", "comment_count", "thumbnail_link", "description"]]
canada_one.info()

In [ ]:
# drop all null values

canada_one = canada_one.dropna()

In [ ]:
# rename columns 

canada_one = canada_one.rename(columns={"publishedAt": "published_at",
                                      "channelTitle": "channel_title",
                                      "categoryId": "category_id"
                                     })

In [ ]:
# change format of published date column so we can work with it

canada_one["published_at"] = pd.to_datetime(canada_one["published_at"])
canada_one['published_at'] = canada_one["published_at"].apply(lambda x: x.strftime('%d-%m-%Y'))

In [ ]:
canada_one

In [ ]:
# grouped data on 'video_id' to collect only the latest date's data

canada_df = canada_one.groupby('video_id').last()
canada_df

In [ ]:
# check data types

canada_df.dtypes


In [ ]:
category_df.dtypes

In [ ]:
# change 'category_id' into int64

category_df['category_id'] = category_df['category_id'].astype("int64")

In [ ]:
# confirm change

category_df.dtypes

In [ ]:
# merge two dataframes together on 'category_id'

merged_df = pd.merge(canada_df , category_df, how = 'inner', on = 'category_id' )
merged_df

In [ ]:
# create a grouped dataframe that shows sum of videos, view counts, likes, dislikes, comment counts of each video category


data_one = merged_df.groupby(['category_title', 'category_id']).sum()
data_one = data_one.rename(columns = {
                                                      "view_count":"total_view_count",
                                                      "likes": "total_likes",
                                                      "dislikes": "total_dislikes",
                                                      "comment_count": "total_comment_count"})
data_one = data_one.reset_index()

data_two = merged_df.groupby('category_title').sum()
data_two = data_two.reset_index()
data_two = data_two[["category_title", 'category_id']]
data_two = data_two.rename(columns={"category_id": "number_of_videos"})

category_data = pd.merge(data_one, data_two, on="category_title")
category_data = category_data.drop(columns='category_title', axis=1)
category_data


In [ ]:
# merge original video dataframe with category_df

merged = pd.merge(canada_one , category_df, how = 'inner', on = 'category_id' )
merged

In [ ]:
# create a video_category dataframe to serve as junction table

video_category = merged[['video_id','category_id']]
video_category

# Load DataFrames into SQL

We then load all of our dataframes into postgres sql. We chose to go with SQL because our data is relational, not too large and is structured and we anticipate that the analysis portion to come will require fast analytical queries and joining of some of the tables. 

In [ ]:
# set up connection to sql

engine = create_engine("postgresql://yingfeng:summer.0@/youtube_db")

In [ ]:
# check all the tables created in sql

engine.table_names()

In [ ]:
# load in all the table data

category_df.to_sql(name='category_df', con=engine, if_exists='append', index=False)

In [ ]:
category_data.to_sql(name='category_data', con=engine, if_exists='append', index=False)

In [ ]:
canada_df.to_sql(name='canada_df', con=engine, if_exists='append', index=True)

In [ ]:
video_category.to_sql(name='video_category', con=engine, if_exists='append', index=False)